In [ ]:
!pip install -q pyomo
from pyomo.environ import * 
import numpy as np
import pandas as pd
!apt-get install -y -qq coinor-cbc

In [2]:
coef = np.loadtxt('lab4_ex1_coef.txt', delimiter=',')

model=ConcreteModel()

N=coef.shape[1]-1
M=coef.shape[0]-1

obj_coef = coef[0,:-1]
constr_coef = coef[1:,:-1]

constr_rhs = coef[1:,-1]

col_indices = np.arange(N)

model.x = Var(col_indices,domain=NonNegativeReals)

row_indices = np.arange(M)

model.constraints = ConstraintList()

for i in row_indices:
  model.constraints.add(summation(constr_coef[i],model.x) <= constr_rhs[i])

model.objective = Objective(expr = summation(obj_coef,model.x), sense=maximize)

model.pprint()

opt_cbc = SolverFactory('cbc')

result = opt_cbc.solve(model)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
       

**Ex1.7** The solver termination condition is unbounded and status is warning. This has happened because the problem we are trying to solve has a unbounded feasible region and is and hence the objective function is taking value infinity hence the warning status is obtained.

The remedy for this situation is that I changed the sense of objective function to minimisation.

In [4]:
#changing it to minimisation problem and then solving  
model.objective.set_sense(minimize)

opt_cbc = SolverFactory('cbc')

result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()

Solver status: ok
Solver termination condition: optimal

Objective =  -17.2125

Decision Variables
x[ 0 ] =  17.925
x[ 1 ] =  1.8
x[ 2 ] =  1.2
x[ 3 ] =  0.0
x[ 4 ] =  0.0
x[ 5 ] =  0.0

Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :  None : 12.000000000000002 :  12.0
      2 :  None : 0.6000000000000001 :   0.6
      3 :  None :                1.2 :   1.2
      4 :  None :            15.2025 :  45.0
      5 :  None :            18.8325 :  27.0


**Ex 1.9** The value of objective function and decision variables are printed above.

Constraints 1,2 and 3 active.  
Constraints 4 and 5 are inactive.